In [40]:
import psycopg2 as pg
import pandas as pd
import os
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

conn = pg.connect(os.getenv('AI_DB_URL'))
open_ai_key = os.environ["OPENAI_API_KEY"]

def fetch_models():
    try:
        cur = conn.cursor()
        query = '''
            SELECT * FROM openai_list_models(_api_key=>%s) ORDER BY created DESC
        '''
        cur.execute(query, (open_ai_key,))
        return cur.fetchall()
    except Exception as e:
        print('Error in fetch_models: {}'.format(e))   
        
def build_model_df(model):
    try:
        # print(model)
        d = {'id': model[0], 'created': model[1], 'owned_by': model[2]}
        df = pd.DataFrame(data=d, index=[0])
        return df
    except Exception as e:
        print('Error in build_model_df: {}'.format(e))   
        
def build_xcel(old_df, path):
    try:
        df = old_df.copy(deep=True)
        df['created'] = df['created'].dt.tz_localize(None)
        df['created'] = df['created'].dt.strftime('%Y-%m-%dT%H%:M%:%S')
        filepath = Path('{}/open_ai_models_{}.xlsx'.format(path, datetime.now().strftime('%Y-%m-%dT%H%M%S')))
        df.to_excel(filepath, index=False)
        return filepath
    except Exception as e:
        print('Error in build_csv: {}'.format(e))     

def main():
    try:
        path = 'data/output/open_ai_models'
        Path('{}'.format(path)).mkdir(parents=True, exist_ok=True)
        models = fetch_models()
        for model in models:
            model_df = build_model_df(model)
            if models.index(model) == 0:
                models_df = model_df
            else:
                models_df = pd.concat([models_df, model_df])
        build_xcel(models_df, path)
        return models, model_df, models_df
    except Exception as e:
        print('Error in main: {}'.format(e))   
    
ret = main()
    
conn.close()